In [93]:
import mysql.connector
import pandas as pd


In [94]:
import os

In [95]:

connection = mysql.connector.connect(
    host='localhost',   
    user='root',    
    password='root', 
    database='Target_Customers'  
)

In [96]:
cursor = connection.cursor()

In [108]:
def create_table_for_country(country):
    table_name = f'Table_{country}'
    
    
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        Customer_Name VARCHAR(255),
        Customer_Id VARCHAR(18),
        Open_Date DATE,
        Last_Consulted_Date DATE,
        Vaccination_Type CHAR(5),
        Doctor_Consulted CHAR(255),
        State CHAR(5),
        Country CHAR(5),
        Post_Code INT,
        Date_Of_Birth DATE,
        Active_Customer CHAR(1)
    );
    """
    
    cursor.execute(create_table_query)
    print(f"Table {table_name} created successfully!")

In [109]:
input_directory = r'C:\Users\Ashmit\Desktop\etlnew\country'

In [110]:
csv_files = [f for f in os.listdir() if f.endswith('.csv')]
os.listdir() 

['.git',
 'country',
 'customers_AUS_20241009_225759.csv',
 'customers_AUS_20241009_230109.csv',
 'customers_BRA_20241009_225759.csv',
 'customers_BRA_20241009_230109.csv',
 'customers_BR_20241009_225759.csv',
 'customers_BR_20241009_230109.csv',
 'customers_data.csv',
 'customers_DE_20241009_225759.csv',
 'customers_DE_20241009_230109.csv',
 'customers_GB_20241009_225759.csv',
 'customers_GB_20241009_230109.csv',
 'customers_GER_20241009_225759.csv',
 'customers_GER_20241009_230109.csv',
 'customers_IND_20241009_225759.csv',
 'customers_IND_20241009_230109.csv',
 'customers_IN_20241009_225759.csv',
 'customers_IN_20241009_230109.csv',
 'customers_JPN_20241009_225759.csv',
 'customers_JPN_20241009_230109.csv',
 'customers_UK_20241009_225759.csv',
 'customers_UK_20241009_230109.csv',
 'customers_USA_20241009_225759.csv',
 'customers_USA_20241009_230109.csv',
 'database_con.ipynb',
 'database_entry.ipynb',
 'readme.md']

In [111]:
countries = set()

In [112]:
for csv_file in csv_files:
    # Read the CSV file
    df = pd.read_csv(csv_file, encoding='ISO-8859-1' )
    
    # Add unique countries to the set
    countries.update(df['Country'].unique())

In [113]:
for country in countries:
    create_table_for_country(country)

Table Table_JPN created successfully!
Table Table_UK created successfully!
Table Table_IND created successfully!
Table Table_AUS created successfully!
Table Table_USA created successfully!
Table Table_BRA created successfully!
Table Table_BR created successfully!
Table Table_GB created successfully!
Table Table_GER created successfully!
Table Table_DE created successfully!
Table Table_IN created successfully!


In [114]:
connection.commit()

In [115]:
# import os
# import pandas as pd

# def insert_data_from_csv(country, input_directory):
#     # Construct the filename pattern based on the country (ignoring timestamp)
#     file_pattern = f'customers_{country}_*.csv'  # Use wildcard for timestamp
#     matching_files = [f for f in os.listdir(input_directory) if f.startswith(f'customers_{country}_') and f.endswith('.csv')]
    
#     # Check if matching files are found
#     if not matching_files:
#         print(f"No files found for country: {country}")
#         return

#     for csv_file in matching_files:
#         # Construct the full path to the CSV file
#         full_csv_path = os.path.join(input_directory, csv_file)
        
#         # Read the CSV file
#         df = pd.read_csv(full_csv_path)

#         # Prepare insert statement
#         for index, row in df.iterrows():
#             insert_query = f"""
#             INSERT INTO Table_{country} (Customer_Name, Customer_ID, Open_Date, Last_Consulted_Date, Vaccination_Type, 
#                                           Doctor_Consulted, State, Country, Post_Code, Date_Of_Birth, Active_Customer)
#             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
#             """

#             # Execute the insert statement with the row data
#             cursor.execute(insert_query, tuple(row))
#         print(f"Data from {full_csv_path} inserted into Table_{country}.")



In [116]:

def insert_or_update_data_from_csv(country, input_directory):
   
    file_pattern = f'customers_{country}_*.csv'  
    matching_files = [f for f in os.listdir(input_directory) if f.startswith(f'customers_{country}_') and f.endswith('.csv')]
    
    
    if not matching_files:
        print(f"No files found for country: {country}")
        return

    for csv_file in matching_files:
        
        full_csv_path = os.path.join(input_directory, csv_file)
        
     
        df = pd.read_csv(full_csv_path)
        
        
        print("Column names in CSV file:", df.columns.tolist())
        
        
        df.columns = df.columns.str.strip()
        if 'Customer_ID' not in df.columns:
            print(f"'Customer_ID' column not found in {csv_file}. Please check the file.")
            return  # 
        
        for index, row in df.iterrows():
            customer_id = row['Customer_ID']  
            
            check_query = f"""
            SELECT Country FROM Table_{country} WHERE Customer_ID = %s;
            """
            cursor.execute(check_query, (customer_id,))
            result = cursor.fetchone()

            if result:
                current_country = result[0]

            
                if current_country != country:
                    # Insert the new record into the new country's table
                    insert_query = f"""
                    INSERT INTO Table_{country} (Customer_Name, Customer_ID, Open_Date, Last_Consulted_Date, Vaccination_Type, 
                                                  Doctor_Consulted, State, Country, Post_Code, Date_Of_Birth, Active_Customer)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                    """
                    cursor.execute(insert_query, tuple(row))
                    print(f"Moved Customer_ID: {customer_id} to Table_{country}.")

                
                    delete_query = f"""
                    DELETE FROM Table_{current_country} WHERE Customer_ID = %s;
                    """
                    cursor.execute(delete_query, (customer_id,))
                    print(f"Deleted Customer_ID: {customer_id} from Table_{current_country}.")
                else:
                    
                    update_query = f"""
                    UPDATE Table_{country}
                    SET Last_Consulted_Date = %s
                    WHERE Customer_ID = %s;
                    """
                    cursor.execute(update_query, (row['Last_Consulted_Date'], customer_id))
                    print(f"Updated Last_Consulted_Date for Customer_ID: {customer_id} in Table_{country}.")
            else:
                
                insert_query = f"""
                INSERT INTO Table_{country} (Customer_Name, Customer_ID, Open_Date, Last_Consulted_Date, Vaccination_Type, 
                                              Doctor_Consulted, State, Country, Post_Code, Date_Of_Birth, Active_Customer)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                """
                cursor.execute(insert_query, tuple(row))
                print(f"Inserted new record for Customer_ID: {customer_id} in Table_{country}.")

    
    connection.commit()





In [117]:
for country in countries:
    insert_or_update_data_from_csv(country, input_directory)

Column names in CSV file: ['Customer_Name', 'Customer_Id', 'Open_Date', 'Last_Consulted_Date', 'Vaccination_Type', 'Doctor_Consulted', 'State', 'Country', 'Post_Code', 'Date_of_Birth', 'Active_Customer']
'Customer_ID' column not found in customers_JPN_20241009_215423.csv. Please check the file.
Column names in CSV file: ['Customer_Name', 'Customer_Id', 'Open_Date', 'Last_Consulted_Date', 'Vaccination_Type', 'Doctor_Consulted', 'State', 'Country', 'Post_Code', 'Date_of_Birth', 'Active_Customer']
'Customer_ID' column not found in customers_UK_20241009_215423.csv. Please check the file.
Column names in CSV file: ['Customer_Name', 'Customer_Id', 'Open_Date', 'Last_Consulted_Date', 'Vaccination_Type', 'Doctor_Consulted', 'State', 'Country', 'Post_Code', 'Date_of_Birth', 'Active_Customer']
'Customer_ID' column not found in customers_IND_20241009_215423.csv. Please check the file.
Column names in CSV file: ['Customer_Name', 'Customer_Id', 'Open_Date', 'Last_Consulted_Date', 'Vaccination_Type'